In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)

In [32]:
from sklearn.model_selection import train_test_split

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.7)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [33]:
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(49000, 784)
(49000,)
(10500, 784)
(10500,)
(10500, 784)
(10500,)


(None, None)

### Random forest classifier

In [43]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [44]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)

rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [45]:
accuracy_score(y_test, y_pred_rf)

0.8208571428571428

### ExtraTree classifier

In [46]:
from sklearn.ensemble import ExtraTreesClassifier

extr_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)

extr_clf.fit(X_train, y_train)

y_pred_extr = extr_clf.predict(X_test)

In [47]:
accuracy_score(y_test, y_pred_extr)

0.810952380952381